In [1]:
'''
Importação do repositório
'''
!git clone https://github.com/teodororo/rna-atividade-avaliativa.git

Cloning into 'rna-atividade-avaliativa'...
remote: Enumerating objects: 2466, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2466 (delta 2), reused 5 (delta 1), pack-reused 2459
Receiving objects: 100% (2466/2466), 59.67 MiB | 18.53 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [2]:
'''
Importação de bibliotecas
'''
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import EfficientNetB7,preprocess_input as b7_preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import SGD

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from itertools import islice
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
'''
Variaveis constantes
'''
TEST_PATH = '/kaggle/working/rna-atividade-avaliativa/dataset_partition/test/'
TRAIN_PATH = '/kaggle/working/rna-atividade-avaliativa/dataset_partition/train/'
VAL_PATH = '/kaggle/working/rna-atividade-avaliativa/dataset_partition/validation/'
num_classes = 15
img_size = (224,224)

'''
Hiperparâmetros 
'''
batch_size = 6 # preferencialmente um multiplo de 2
weights = 'imagenet' # pode ser None (pesos aleatórios)
trainable_layers = False # se for True, ele ajusta tudo
epocas = 20

'''
Hiperparâmetros da camada extra do output
'''
_optimizer = keras.optimizers.Adam()
_activation = 'softmax'

In [4]:
'''
Ajuste da base de dados
'''

train_datagen = ImageDataGenerator(preprocessing_function=b7_preprocess_input,rescale=1./255)
val_datagen = ImageDataGenerator(preprocessing_function=b7_preprocess_input,rescale=1./255)
test_datagen = ImageDataGenerator(preprocessing_function=b7_preprocess_input,rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
        VAL_PATH,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        TEST_PATH,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical')

Found 1648 images belonging to 15 classes.
Found 239 images belonging to 15 classes.
Found 448 images belonging to 15 classes.


In [5]:
def create_model():
    base_model = EfficientNetB7(
        input_shape=img_size + (3,),
        include_top=False,
        weights=weights
    )
    base_model.trainable = False

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes, activation=_activation)
    ])

    return model

model = create_model()
model.compile(
    optimizer=_optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

258076736/258076736 [==============================] - 11s 0us/step


In [ ]:
#checkpoint = ModelCheckpoint('best_weights.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epocas,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    #callbacks=[checkpoint]
)

Epoch 1/200


2023-03-13 08:02:51.191287: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetb7/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


275/275 [==============================] - 50s 92ms/step - loss: 2.8547 - accuracy: 0.0716 - val_loss: 2.8134 - val_accuracy: 0.0753
Epoch 2/200
275/275 [==============================] - 20s 71ms/step - loss: 2.8363 - accuracy: 0.0704 - val_loss: 2.8322 - val_accuracy: 0.0544
Epoch 3/200
275/275 [==============================] - 19s 70ms/step - loss: 2.8402 - accuracy: 0.0692 - val_loss: 2.8954 - val_accuracy: 0.0753
Epoch 4/200
275/275 [==============================] - 20s 71ms/step - loss: 2.8051 - accuracy: 0.0783 - val_loss: 2.8786 - val_accuracy: 0.0628
Epoch 5/200
275/275 [==============================] - 22s 80ms/step - loss: 2.8133 - accuracy: 0.0783 - val_loss: 2.7603 - val_accuracy: 0.0962
Epoch 6/200
275/275 [==============================] - 20s 71ms/step - loss: 2.8180 - accuracy: 0.0777 - val_loss: 2.7637 - val_accuracy: 0.0837
Epoch 7/200
275/275 [==============================] - 20s 72ms/step - loss: 2.8047 - accuracy: 0.0819 - val_loss: 2.8326 - val_accuracy: 0.07

In [ ]:
test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
predictions = model.predict(test_generator, steps=test_steps_per_epoch)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_labels,zero_division=0)
print(report)

In [ ]:
true_classes = test_generator.classes
test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
predictions = model.predict(test_generator, steps=test_steps_per_epoch)
predicted_classes = np.argmax(predictions, axis=1)
conf_matrix = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10,8))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt='g', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Classe prevista')
plt.ylabel('Classe verdadeira')
plt.title('Matriz de confusão')
plt.show()